In [1]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.isri import ISRIStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# dftrain = pd.read_csv("train_arabic.csv", encoding = "utf16")
dftrain = pd.read_excel("6.xlsx",engine="openpyxl")
# dftrain.drop(columns=['Unnamed: 10', 'Unnamed: 11' , 'Unnamed: 12' , 'tags' , 'selfeval'] , inplace = True)
dftrain.dropna(axis = 0, inplace = True)
# dftrain = dftrain[:5000]
X, y = dftrain.translated_utterance, dftrain.context

In [3]:
for letter in '#.][!X،R؟'+'''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''':
    X = X.astype(str).str.replace(letter,'')    
documents = []
for text in X:
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)  
    documents.append(text)

In [4]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [5]:
len(X)

5475

In [6]:
len(documents)

5475

In [8]:
stopwords_list = stopwords.words('arabic')
stopwords_list = stopwords_list + ['ا','لقد','كنت','كان']
listToStr = ' '.join([str(elem) for elem in stopwords_list]) 
documents =pd.Series(documents)
# documents=documents.apply(lambda x: [item for item in x if item not in stopwords_list])

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

unigramdataGet= word_vectorizer.fit_transform(documents)
unigramdataGet = unigramdataGet.toarray()

vocab = word_vectorizer.get_feature_names()
unigramdata_features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
unigramdata_features[unigramdata_features>0] = 1

unigramdata_features.head()

,10,100,1000,11,1130,12,1200,122,1264conv,1265conv,1266conv,1267conv,1268conv,1269conv,13,14,140,147,15,150,16,160,17,18,180,187,1937,1947,194conv,195conv,196conv,197conv,198conv,1991,1995,199792,199793,199794,1998,199811,...,ينفقون,ينقطع,ينمو,ينهار,ينهي,يهاجمني,يهتم,يهتمون,يهداوا,يهربوا,يهزك,يهم,يهمون,يهنيوا,يواصل,يوافق,يوتي,يوتيوب,يوثر,يوجد,يوخذ,يوخرني,يودوا,يودون,يودي,يوذي,يوسفني,يوفر,يوقظونني,يوقعها,يوك,يولم,يوم,يوما,يومك,يومي,يوميا,يومين,يوني,ييكيس
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X=unigramdata_features

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [15]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
clf.fit(X, y)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=False, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [30]:
def predict(sentence):
    documents =[sentence]
    X=word_vectorizer.transform(documents).toarray()
    return clf.predict(X)

In [20]:
predict("يدفعني للجنون")

hena


array(['annoyed'], dtype=object)

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import torch
torch.cuda.empty_cache()
!pip install -U transformers==4.5.1
import torch.nn as nn
from transformers import pipeline

chatbot = pipeline("text2text-generation",model='/content/drive/MyDrive/arabert2arabert')

     |████████████████████████████████| 2.1MB 7.7MB/s 
     |████████████████████████████████| 3.3MB 28.3MB/s 
     |████████████████████████████████| 901kB 24.6MB/s 


In [24]:
chatbot("تنتهي اجازة الصيف",
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[{'generated_text': 'أوه ، هذا يبدو وكأنه عطلة نهاية الأسبوع.'}]

In [27]:
def NLP(S):
  print(chatbot(S,
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3))
  print(predict(S))
  


In [38]:
NLP("شعرت بالفخر عندما حققت حلما")

[{'generated_text': 'أنا سعيد لأنك عملت بشكل جيد.'}]
['proud']


In [39]:
NLP("انا منزعج من زميل")

[{'generated_text': 'أوه لا! ماذا حدث ؟'}]
['proud']


In [40]:
NLP("اخيرا انجزت بعض اعمال الفناء")

[{'generated_text': 'أوه ، واو! يجب أن تكون فخورة جدا.'}]
['proud']


In [41]:
NLP("اثناء الكلية , تعرضت للتنمر")

[{'generated_text': 'أوه لا! ماذا حدث ؟'}]
['annoyed']


In [42]:
NLP("صديقي يزعجني")

[{'generated_text': 'انا اسف لسماع ذلك.'}]
['annoyed']


In [43]:
NLP("صديقي نرجسي في بعض الاحيان")

[{'generated_text': 'انا اسف لسماع ذلك.'}]
['proud']


In [44]:
NLP("تمكنت من انقاذ سيدة عمياء")

[{'generated_text': 'أوه ، واو! أنا آسف جدا لسماع ذلك.'}]
['proud']


In [45]:
NLP("انا فخور جدا بزوجتي")

[{'generated_text': 'أوه ، واو! كم عمرها ؟'}]
['proud']


In [46]:
NLP("كنت ابذل مجهودا جماعيا , و اذا كان فريقي يعمل بشكل جيد")

[{'generated_text': 'أنا متأكد من أنك ستبلي بلاء حسنا!'}]
['proud']


In [47]:
NLP("سدد ابن اخي الكرة بعيدا جدا في مباراة البيسبول في ذلك اليوم")

[{'generated_text': 'أوه ، واو! هل أنت بخير ؟'}]
['proud']
